In [13]:
#manipulate dataframes in python
import pandas as pd

#make API calls with python
import requests

import localvars

#allows us to store results of API call cleanly
import json

#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('../src/data/laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)

#construct the API call we will use
baseAPI = "https://api.census.gov/data/2017/acs/acs5?get=B01003_001E&for=tract:*&in=state:47%20county:125&key=" 
            
calledAPI = baseAPI + localvars.apiKey

#call the API and collect the response
response = requests.get(calledAPI)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#flip the order of the response from [population, zipcode] -> [zipcode, population]
formattedResponse = [item[::-1] for item in formattedResponse]

#store the response in a dataframe
metric = pd.DataFrame(columns=['id', '2', '3', 'y_2017'], data=formattedResponse)
del metric['2']
del metric['3']

#save that dataframe to a CSV spreadsheet
metric.to_csv('laZipPopulations.csv', index=False)

